## Build topic model

In [14]:
import pandas as pd

In [25]:
n_features = 5000 # number of most common words
n_topics = 30 # number of topics
n_top_words = 60 # number of characteristic words per topic (viz purposes only)
max_df = 0.5 # maximum document frequency
min_df = 100 # minimum document frequency
max_books = 1000 # include all books
chunk_size = 300 # size of average Harry Potter chapter (after keeping only ADJ, NOUN & VERB)

In [26]:
import glob
import random
RND = 12345
random.seed(RND)

class Chunker(object):
    def __init__(self, filenames, max_chars_per_book=None,
                 chunk_size=None):
        self.max_chars_per_book = max_chars_per_book
        self.chunk_size = chunk_size
        self.filenames = filenames

    def __iter__(self):
        for filename in self.filenames:
            with open(filename) as f:
                text = f.read()
                if self.max_chars_per_book:
                    text = text[:self.max_chars_per_book]
            
            tokens = text.split()
            
            for i in range(0, len(tokens), self.chunk_size):
                yield tokens[i:i + self.chunk_size]

In [33]:
path = 'data/tagged/streek/*.txt'
filenames = sorted(list(glob.glob(path)))
print(len(filenames))

50


In [34]:
chunker = Chunker(filenames, chunk_size=chunk_size)

In [35]:
import os
from sklearn.feature_extraction.text import CountVectorizer

def identity(x):
    return x

vectorizer = CountVectorizer(max_df=max_df, min_df=min_df,
                             max_features=n_features,
                             analyzer=identity)
X = vectorizer.fit_transform(chunker)

bow_example = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
bow_example.sample(10)

,a,aandacht,aandachtig,aandoen,aangeboden,aangekeken,aangekomen,aangenomen,aankijken,aankomen,...,zwarte,zweeg,zweet,zwijgen,zwijgend,zwijgt,zélf,zó,één,óók
3507,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4835,0,0,0,0,0,0,0,0,0,0,...,7,0,0,0,0,0,0,0,2,0
3441,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
355,1,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,3,0
1004,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5821,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2364,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5579,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1267,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
957,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
X_ = transformer.fit_transform(X)

In [37]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=n_topics,
          random_state=RND,
          verbose=1, max_iter=n_topics).fit(X_)

violation: 1.0
violation: 0.41793887036448085
violation: 0.29592196620678946
violation: 0.19250065306166914
violation: 0.11648847001109655
violation: 0.07762911873129312
violation: 0.05654371161575821
violation: 0.04502088246039263
violation: 0.037637597756080145
violation: 0.032455858643444864
violation: 0.02872612589952708
violation: 0.025917065664696714
violation: 0.023594134492937728
violation: 0.021557542610322614
violation: 0.019719252901369763
violation: 0.018221832777507045
violation: 0.017084994851341836
violation: 0.01624826056614422
violation: 0.01582424037230903
violation: 0.01569538784951964
violation: 0.015767322728756313
violation: 0.015676211596075126
violation: 0.01537315648158281
violation: 0.014708426603159136
violation: 0.013773992748037674
violation: 0.01289684069805528
violation: 0.012338362067158617
violation: 0.011915771562779594
violation: 0.011600628940723182
violation: 0.011259792606965522


In [38]:
import os
import shutil
from wordcloud import WordCloud
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

def top_words(model, feature_names, n_top_words):
    try:
        shutil.rmtree('clouds')
    except:
        pass
    os.mkdir('clouds')

    for topic_idx, topic in enumerate(model.components_):
        print('.', end='')
        topic[np.isnan(topic)] = 0
        
        words = [feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]]
        weights = [topic[i] for i in topic.argsort()[:-n_top_words-1:-1]]

        freqs = {wo: we for wo, we in zip(words, weights)}
        wordcloud = WordCloud(normalize_plurals=False,
                              background_color='white',
                              colormap='inferno_r',
                              width=800,
                              height=400)
        wordcloud = wordcloud.generate_from_frequencies(freqs)
        wordcloud.to_file('clouds/'+str(topic_idx) + '.tiff')

feature_names = vectorizer.get_feature_names()
info = top_words(nmf, feature_names, n_topics)

..............................

In [41]:
chunks = Chunker(filenames, chunk_size=chunk_size, max_chars_per_book=None)
X = vectorizer.transform(chunks)

In [42]:
import pyLDAvis
pyLDAvis.enable_notebook()
import pyLDAvis.sklearn

In [43]:
nmf.verbose = 0
pyLDAvis.sklearn.prepare(nmf, X, vectorizer)

/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0      0.098873 -0.050018       1        1  8.843821
25    -0.122180  0.073100       2        1  8.819586
24    -0.172698  0.048729       3        1  8.425097
17    -0.004958 -0.022761       4        1  7.809198
1      0.241067  0.222053       5        1  4.636178
27    -0.060855  0.007865       6        1  4.496855
2     -0.142245  0.033542       7        1  3.827836
9      0.056186 -0.011058       8        1  3.790332
10    -0.058832 -0.037107       9        1  3.725110
26     0.110179  0.093957      10        1  3.253412
16     0.063706 -0.023611      11        1  3.157708
4     -0.041058  0.138516      12        1  2.981682
23    -0.022701 -0.145454      13        1  2.845400
3     -0.145252  0.060285      14        1  2.769253
13    -0.123974 -0.003092      15        1  2.553328
19     0.149870  0.026003      16        1  2.510243
8      0.091567  0.149418      17        1  2.340668
12     0.066059 -0.177816      18        1  2.134242
6      0.066725 -0.066651      19        1  2.119541
29     0.002275 -0.042333      20        1  2.053412
7      0.074064 -0.107144      21        1  2.028783
22     0.060071  0.122211      22        1  1.984341
18     0.102239 -0.160735      23        1  1.876793
11     0.055993 -0.067194      24        1  1.834215
28    -0.050189 -0.016381      25        1  1.803203
21    -0.030719 -0.038947      26        1  1.710730
14    -0.171641  0.024280      27        1  1.614615
20    -0.048579 -0.071118      28        1  1.438383
15     0.062653 -0.018874      29        1  1.371199
5     -0.105645  0.060336      30        1  1.244837, topic_info=        Term         Freq        Total Category  logprob  loglift
1815   vader  6558.000000  6558.000000  Default  30.0000  30.0000
735        h  5361.000000  5361.000000  Default  29.0000  29.0000
349   dokter  4044.000000  4044.000000  Default  28.0000  28.0000
533    geert  3341.000000  3341.000000  Default  27.0000  27.0000
1734   tante  3280.000000  3280.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
1652   staat    57.115989  2155.391855  Topic30  -5.4759   0.7555
878     jaar    55.203937  2038.764746  Topic30  -5.5100   0.7771
934     keer    52.117645  1730.629282  Topic30  -5.5675   0.8834
1374    oude    52.405178  1961.681771  Topic30  -5.5620   0.7636
1052    laat    50.733878  2072.755805  Topic30  -5.5944   0.6761

[1632 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         2  0.008137    a
0         4  0.017179    a
0         5  0.686711    a
0         7  0.033002    a
0         8  0.000452    a
...     ...       ...  ...
2231     10  0.164821  óók
2231     13  0.123142  óók
2231     20  0.126931  óók
2231     21  0.018945  óók
2231     26  0.049257  óók

[14861 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 26, 25, 18, 2, 28, 3, 10, 11, 27, 17, 5, 24, 4, 14, 20, 9, 13, 7, 30, 8, 23, 19, 12, 29, 22, 15, 21, 16, 6])